In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas

from shared_code.utility.spark.set_environ import set_azure_env

set_azure_env()

from shared_code.utility.storage.table import TableAdapter

In [2]:
table_name = "training"

table_adapter: TableAdapter = TableAdapter()

data_path = "/data/parquet/"

parquet_process_data_path = data_path + "processed_data.parquet"

In [3]:
print(f"Reading from parquet {parquet_process_data_path} with Updated Primary Captions")

processed_with_captions = pandas.read_parquet(parquet_process_data_path)

display(processed_with_captions)

Reading from parquet /data/parquet/processed_data.parquet with Updated Primary Captions


,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...,a collage of photos showing a man in a suit an...
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a nighttime view of a large rock formation,a nighttime view of a large rock formation
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a boat sitting on top of a lush green field,a boat sitting on top of a lush green field
...,...,...,...,...,...,...,...,...,...,...,...
11724,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,a large amount of light is reflected in the sky,a large amount of light is reflected in the sky
11725,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a blue and white photo of a blue and white obj...,a blue and white photo of a blue and white obj...
11726,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...,A man wearing a black shirt and black pants is...
11728,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red shirt and a red hat,a man in a red shirt and a red hat


In [4]:
from typing import List
from dataclasses import dataclass

@dataclass
class DataSources:
	name: str
	data: List[str]

	@staticmethod
	def from_dict(obj: dict) -> 'DataSources':
		_name = obj.get("name")
		_data = [x for x in obj.get("data")]
		return DataSources(_name, _data)

In [5]:
sources = [
	{"name": "CityDiffusion", "data": [
		"CityPorn"
	]},
	{"name": "NatureDiffusion",
	 "data": [
		 "EarthPorn"
	 ]},
	{"name": "CosmicDiffusion",
	 "data": [
		 "SpacePorn"
	 ]},
	{"name": "MemeDiffusion",
	 "data": [
		 "greentext",
		 "memes"
	 ]},
	{"name": "SexyDiffusion",
	 "data": [
		 "sfwpetite",
		 "selfies",
		 "Amicute",
		 "amihot",
		 "AmIhotAF",
		 "HotGirlNextDoor",
		 "SFWNextDoorGirls",
		 "SFWRedheads"]
	 }
]

data_sources = [DataSources.from_dict(x) for x in sources]

In [20]:
foo = processed_with_captions.where(lambda x: x.subreddit.isin([
		 "sfwpetite",
		 "selfies",
		 "Amicute",
		 "amihot",
		 "AmIhotAF",
		 "HotGirlNextDoor",
		 "SFWNextDoorGirls",
		 "SFWRedheads"])).dropna()
display(foo)

lines = []
for record in foo.to_dict(orient='records'):
	subreddit = record['subreddit']
	for source in data_sources:
		if subreddit in source.data:
			line = f"<|startoftext|><|model|>{source.name}<|prompt|>{record['text']}<|text|>{record['original_caption']}<|endoftext|>" + "\n"
			lines.append(line)
with open("training.txt", "wb") as f:
	for line in lines:
		f.write(line.encode("utf-8"))

,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
9,sfwpetite,v3ra9g4vrb9a1.jpg,Braids,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,True,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,True,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,a beautiful young woman in a bikini posing for...,a beautiful young woman in a bikini posing for...
12,sfwpetite,ldz8tgpsub9a1.jpg,Incredible body,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,True,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,True,a0b0ffef01a7abf9de351000026d49f0,1001ssd,a woman in a bikini standing on a beach,a woman in a bikini standing on a beach
13,SFWRedheads,qlr6w888ga9a1.jpg,I heard wearing red on NYE brings good luck......,D:\data\images\SFWRedheads\thumbnail\qlr6w888g...,True,D:\data\images\SFWRedheads\qlr6w888ga9a1.jpg,True,5c64d4d6a1b12586af2e4499dc746bf3,100252z,a beautiful young woman in a bikini posing for...,a beautiful young woman in a bikini posing for...
17,SFWRedheads,el3s490lzb9a1.jpg,Happy New Year to every single person near and...,D:\data\images\SFWRedheads\thumbnail\el3s490lz...,True,D:\data\images\SFWRedheads\el3s490lzb9a1.jpg,True,1fb785ca16b10f4f7613961f0b88f369,1002cx2,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
...,...,...,...,...,...,...,...,...,...,...,...
11706,SFWRedheads,s20oajzpja9a1.jpg,Good morning! What are your plans today?,D:\data\images\SFWRedheads\thumbnail\s20oajzpj...,True,D:\data\images\SFWRedheads\s20oajzpja9a1.jpg,True,8ee5ffd721683d9768d8463a0d455724,zzw558,A woman in a pink dress is standing in front o...,A woman in a pink dress is standing in front o...
11708,HotGirlNextDoor,mrq7di60499a1.jpg,Hotties (IKTR),D:\data\images\HotGirlNextDoor\thumbnail\mrq7d...,True,D:\data\images\HotGirlNextDoor\mrq7di60499a1.jpg,True,d26c8da063c60fa56c075005f847f7e3,zzwcqy,A woman in a bikini and a man in a cowboy hat ...,A woman in a bikini and a man in a cowboy hat ...
11715,HotGirlNextDoor,dozyybyzwa9a1.jpg,Perfect Day (IKTR),D:\data\images\HotGirlNextDoor\thumbnail\dozyy...,True,D:\data\images\HotGirlNextDoor\dozyybyzwa9a1.jpg,True,d273d2f322103b2a431e9414e6443dfc,zzxq2l,A girl in a pink dress is standing in front of...,A girl in a pink dress is standing in front of...
11717,SFWNextDoorGirls,Hzad3q2.jpg,your new years resolution should be sleeping w...,D:\data\images\SFWNextDoorGirls\thumbnail\Hzad...,True,D:\data\images\SFWNextDoorGirls\Hzad3q2.jpg,True,d159090b1c7b485347cec61c70bc9c54,zzy3sq,A girl in a pink dress is talking to another g...,A girl in a pink dress is talking to another g...


In [7]:
lines = []
for record in processed_with_captions.to_dict(orient='records'):
	subreddit = record['subreddit']
	for source in data_sources:
		if subreddit in source.data:
			line = f"<|startoftext|><|model|>{source.name}<|prompt|>{record['text']}<|text|>{record['original_caption']}<|endoftext|>" + "\n"
			lines.append(line)
with open("training.txt", "wb") as f:
	for line in lines:
		f.write(line.encode("utf-8"))